In [65]:
#!pip install -U certifi
#!pip install -U elasticsearch
#!pip install -U xmljson

In [67]:
import certifi
import json
import requests
import urllib
import re
import codecs
import xml.dom.minidom
from xmljson import badgerfish as bf
#from elasticsearch import Elasticsearch
from xml.etree.ElementTree import fromstring#, Element, tostring

rmindex = requests.delete("http://es:9200/nawrsdev")
print(rmindex.status_code)
mapping = {}
with open("elastic_mapping_rev.json", 'r') as imap:
    mapping = json.load(imap)
    imap.close()

addindex = requests.put("http://es:9200/nawrsdev", data=json.dumps(mapping))
print(addindex.text)

datalist = []

def oaiItemHarvest(oaiURL, identifier, autoNum):
    params = urllib.parse.urlencode({'verb': 'GetRecord',
                               'metadataPrefix': 'dcs',
                               'identifier': identifier})
    requestURL = oaiURL + "?" + params
    response = urllib.request.urlopen(requestURL)
    result = response.read()
    uRes = codecs.decode(result, 'utf-8')
    datalist.append(bf.data(fromstring(uRes.encode('utf-8'))))
    return

def oaiResume(oaiURL, oaiXML):
    # get resumption token and build new URL
    rToken = oaiXML.getElementsByTagNameNS('*', 'resumptionToken')
    strToken = rToken[0].firstChild.nodeValue
    params = {'verb': 'ListRecords', 'resumptionToken': strToken}
    # remove existing resumptionToken from XML
    ListRecs = oaiXML.getElementsByTagNameNS('*', 'ListRecords')
    ListRecs[0].removeChild(rToken[0])
    # send the request
    response = requests.get(oaiURL, data=params)
    result = response.content
    uRes = codecs.decode(result, 'utf-8')
    rXML = xml.dom.minidom.parseString(uRes.encode("utf-8"))
    # append all returned ListRecords children
    recs = rXML.getElementsByTagNameNS('*', 'record')
    for child in recs:
        ListRecs[0].appendChild(child)
    # see if there is another resumptionToken
    # if so, append to oaiXML and recurse
    rToken2 = rXML.getElementsByTagNameNS('*', 'resumptionToken')
    if len(rToken2) == 1:
        if rToken2[0].firstChild:
            ListRecs[0].appendChild(rToken2[0])
            oaiResume(oaiURL, oaiXML)
    return oaiXML
    

# Request collection level metadata via the dspace OAI-PMH interface
# currently brute-force requesting oai_dc, but __main__ can be easily
# edited to allow users to request metadata formats and select accordingly
def oaiHarvest(oaiURL, getSet):
    oaiColHandle = "output_" + "nawrs"
    params = {'verb': 'ListRecords', 'metadataPrefix': 'dcs', 'set': getSet}
    response = requests.get(oaiURL, data=params)
    #print(response.text)
    result = response.content
    uRes = codecs.decode(result, 'utf-8')
    oaiXML = xml.dom.minidom.parseString(uRes.encode("utf-8"))
    # check if there is a resumptionToken
    # if there is, make a follow up request
    # and append <record> elements from request
    # to existing XML
    # alternatively, we can save all requests/responses
    rToken = oaiXML.getElementsByTagNameNS('*', 'resumptionToken')
    if len(rToken) == 0:
        #with codecs.open(oaiColHandle + '.xml', 'w', encoding='utf-8') as data:
            #oaiXML.writexml(data, indent='', newl='')
        print("got series data")
    else:
        resumeXML = oaiResume(oaiURL, oaiXML)
        #with codecs.open(oaiColHandle + '.xml', 'w', encoding='utf-8') as data:
            #resumeXML.writexml(data, indent='', newl='')
    records = oaiXML.getElementsByTagNameNS('*', 'identifier')
    autoNum = 0
    for record in records:
        value = record.firstChild.nodeValue
        protocol = re.compile('oai')
        getOAI = protocol.match(value)
        if getOAI:
            autoNum = autoNum + 1
            oaiItemHarvest(oaiURL, value, autoNum)
    return

# if dictionary

def mddict(data):
    textdata = data["$"]
    removep1 = textdata.replace('<p>', '')
    removep2 = removep1.replace('</p>', '')
    return removep2
    

# if list, value is a list of dictionaries
# so do a for each and concat as comma separated values

def mdlist(data):
    tempList = []
    for each in data:
        textdata = each["$"]
        removep1 = textdata.replace('<p>', '')
        tempList = removep1.replace('</p>', '')
    return tempList

#es = Elasticsearch("http://es:9200")
mdList = []



oaiURL = "http://digitalrepository.unm.edu/do/oai/"
getSet = "publication:nawrs"
oaiHarvest(oaiURL, getSet)

h1 = "{http://www.openarchives.org/OAI/2.0/}OAI-PMH"
h2 = "{http://www.openarchives.org/OAI/2.0/}GetRecord"
h3 = "{http://www.openarchives.org/OAI/2.0/}record"
h4 = "{http://www.openarchives.org/OAI/2.0/}metadata"
h5 = "{http://www.openarchives.org/OAI/2.0/oai_dc/}dc"

for each in datalist:
    try:
        #jdata = json.load(each)
        md = each[h1][h2][h3][h4][h5]
        mdList.append(md)
    except Exception as e:
        print(e)
        pass

doctitle = {"title": "{http://purl.org/dc/elements/1.1/}title"}
doccreator = {"creator": "{http://purl.org/dc/elements/1.1/}creator"}
doctribes = {"tribes": "{http://purl.org/dc/elements/1.1/}dc.subject.tribes"}
docsettlement = {"settlement": "{http://purl.org/dc/elements/1.1/}dc.subject.settlement"}
doccoverage = {"coverage": "{http://purl.org/dc/elements/1.1/}coverage"}
dochuc = {"huc": "{http://purl.org/dc/elements/1.1/}dc.subject.hydologic_code"}
doctype = {"type": "{http://purl.org/dc/elements/1.1/}type"}
docdate = {"date": "{http://purl.org/dc/elements/1.1/}date"}
dockwds = {"subject": "{http://purl.org/dc/elements/1.1/}subject"}
docdesc = {"description": "{http://purl.org/dc/elements/1.1/}description"}
docpoly = {"polygon": "{http://purl.org/dc/elements/1.1/}dc.coverage.spatial.polygon"}
doccent = {"centroid": "{http://purl.org/dc/elements/1.1/}dc.coverage.spatial.centroid"}
docid = {"identifier": "{http://purl.org/dc/elements/1.1/}identifier"}

textmeta = [doctitle, doccreator, doctype, docdate, docid, doccoverage, dockwds, docdesc]
geometa = [docpoly, doccent]

singleton = []
singleton.append(mdList[0])

for doc in mdList:
    esdoc = {}
    dcid = ''
    for field in textmeta:
        metadata = ""
        try:
            for k, v in field.items():
                if isinstance(doc[v], list):
                    metadata = mdlist(doc[v])
                elif isinstance(doc[v], dict):
                    metadata = mddict(doc[v])
                    if k == "tribes":
                        metadata = [metadata]
                    if k == "identifier":
                        dcid = doc[v]
                esdoc[k] = metadata
        except Exception as e:
            #print(dcid, e, k, v)
            pass
    for geofield in geometa:
        metadata = ""
        try:
            for k, v in geofield.items():
                metadata = mddict(doc[v])
            esdoc[k] = json.loads(metadata)
        except Exception as e:
            #print(dcid, e, k, v)
            pass
    try:
        #es.index(index="nawrsdev", doc_type="item", body=json.dumps(esdoc))
        requests.post("http://es:9200/nawrsdev/item", data=json.dumps(esdoc))
        pass
    except Exception as e:
        print(dcid, e)
        pass

print("done")

200
{"acknowledged":true,"shards_acknowledged":true}
done


In [56]:
#requests.get("http://es:9200/nawrsdev/_stats?pretty").json()
#rmindex = requests.delete("http://es:9200/nawrsdev")
#print(rmindex.status_code)